In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
import optuna
from vacances_scolaires_france import SchoolHolidayDates
from sklearn.model_selection import train_test_split
import importlib

from preprocess_FE import (
    get_zone_c_holidays,
    encode_dates,
    engineer_weather_features, 
    remove_outliers
)

In [2]:
holiday_dates = get_zone_c_holidays()

In [3]:
data = pd.read_parquet(Path("data") / "train.parquet")
data = remove_outliers(data)

important_columns = ["date", "pres", "ff", "t", "vv", "rr1"]
weather_data = pd.read_csv("./external_data/external_data.csv", usecols=important_columns)

/Users/martynaharasym/bike_counters/preprocess_FE.py:196: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])


In [4]:
weather_data["date"] = pd.to_datetime(weather_data["date"])
weather_data = weather_data.dropna(axis=1, how="all")
weather_data.set_index("date", inplace=True)
weather_data = weather_data[~weather_data.index.duplicated(keep="first")]
weather_data_interpolated = weather_data.resample("h").interpolate(method="linear")

In [5]:
columns = ['date', 'nouveaux_patients_hospitalises']
covid_data = pd.read_csv('./synthese-fra (1).csv', parse_dates=False, usecols=columns)
covid_data['date_only'] = pd.to_datetime(covid_data['date']).dt.date

In [6]:
merged_data = data.merge(weather_data_interpolated, on="date", how="left")
merged_data['date_only'] = pd.to_datetime(merged_data['date']).dt.date

merged_data = merged_data.merge(covid_data, on="date_only", how="left")

In [7]:
X = merged_data[["counter_name", "site_name", "date_x", "longitude", "latitude", "ff", "t", "vv", "pres", "rr1", "nouveaux_patients_hospitalises"]]
y = merged_data["log_bike_count"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply date encoding

X_train_encoded1 = encode_dates(X_train, holiday_dates)
X_test_encoded1 = encode_dates(X_test, holiday_dates)

X_train_encoded3 = engineer_weather_features(X_train_encoded1)
X_test_encoded3 = engineer_weather_features(X_test_encoded1)

# Column transformer for preprocessing
categorical_features = ["counter_name", "site_name", "rain_category", "season", "year"
                        # "snow_category"
                       ]
numerical_features = list(X_train_encoded3.drop(columns=categorical_features).columns)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

In [8]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
    }
    
    xgb_model = XGBRegressor(**params, random_state=42)
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", xgb_model)
    ])
    pipeline.fit(X_train_encoded3, y_train)
    y_pred = pipeline.predict(X_test_encoded3)
    rmse = root_mean_squared_error(y_test, y_pred)
    return rmse

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

best_params = study.best_params
best_model = XGBRegressor(**best_params, random_state=42)

xgboost_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", best_model)
])

[I 2024-12-09 15:24:42,157] A new study created in memory with name: no-name-ec2b2e2b-3933-4dfe-bae9-8c3b8e4992ce
[I 2024-12-09 15:24:46,267] Trial 0 finished with value: 0.5700825277728115 and parameters: {'n_estimators': 495, 'max_depth': 3, 'learning_rate': 0.056515057162942896, 'subsample': 0.6869327551250088, 'colsample_bytree': 0.7262339387472905, 'reg_alpha': 7.940536440021034, 'reg_lambda': 4.817454594970283}. Best is trial 0 with value: 0.5700825277728115.
[I 2024-12-09 15:24:52,643] Trial 1 finished with value: 0.4486136091894118 and parameters: {'n_estimators': 267, 'max_depth': 9, 'learning_rate': 0.02815218615852564, 'subsample': 0.7217607501269978, 'colsample_bytree': 0.7072622202896365, 'reg_alpha': 9.832807068835914, 'reg_lambda': 9.827564298936313}. Best is trial 1 with value: 0.4486136091894118.
[I 2024-12-09 15:24:55,467] Trial 2 finished with value: 0.43837663218494877 and parameters: {'n_estimators': 110, 'max_depth': 8, 'learning_rate': 0.10928911654754897, 'subsa

Best hyperparameters: {'n_estimators': 490, 'max_depth': 10, 'learning_rate': 0.12130535895435919, 'subsample': 0.8831961055121214, 'colsample_bytree': 0.7141237725816447, 'reg_alpha': 2.350467921774497, 'reg_lambda': 2.946704645556712}


In [9]:
xgboost_pipeline.fit(X_train_encoded3, y_train)
y_pred = xgboost_pipeline.predict(X_test_encoded3)
final_rmse = root_mean_squared_error(y_test, y_pred)
print(f"Final XGBoost RMSE: {final_rmse:.4f}")

Final XGBoost RMSE: 0.3373


In [10]:
df_test = pd.read_parquet("./data/final_test.parquet")
df_test_merged = df_test.merge(weather_data_interpolated, on='date', how='left')
df_test_merged['date_only'] = pd.to_datetime(df_test_merged['date']).dt.date

df_test_merged = df_test_merged.merge(covid_data, on='date_only', how='left')

X = df_test_merged[["counter_name", "site_name", "date_x", "longitude", "latitude", "ff", "t", "vv", "pres", "rr1", "nouveaux_patients_hospitalises"]]

X = encode_dates(X, holiday_dates)
X = engineer_weather_features(X)

X_test_final = preprocessor.transform(X)
final_predictions = xgboost_pipeline.named_steps['model'].predict(X_test_final)

results = pd.DataFrame({"Id": np.arange(final_predictions.shape[0]), "log_bike_count": final_predictions})
results.to_csv("submission_xgboost_optuna3.csv", index=False)

/Users/martynaharasym/bike_counters/preprocess_FE.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["year"] = X["date_x"].dt.year
/Users/martynaharasym/bike_counters/preprocess_FE.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["month"] = X["date_x"].dt.month
/Users/martynaharasym/bike_counters/preprocess_FE.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati